In [1]:
import numpy as np
import pandas as pd
import yfinance as yf

from scipy.optimize import minimize

In [78]:
# pd.set_option('display.max_rows', 20)

In [22]:
# import yfinance as yf
# import pandas as pd
# import numpy as np
# from scipy import optimize
# 
# import os
# cwd = os.getcwd()
# print(cwd)
# os.chdir(r'C:\Users\mitch\Documents\QCF\Finance and Investments\StockTrack-Trading-Simulation')

C:\Users\mitch\Documents\QCF\Finance and Investments\StockTrack-Trading-Simulation


# Reusable functions

In [6]:
def calculate_return(df):
    if df.empty:
        return None
    # Get the closing price at the start of the year
    start_price = df['Close'].iloc[0]

    # Get the closing price at the end of the year
    end_price = df['Close'].iloc[-1]

    # Calculate the one-year return
    return (end_price - start_price) / start_price
# 
# 
# def MaximizeSharpeRatioOptmzn(MeanReturns, CovarReturns, RiskFreeRate, PortfolioSize):
#     # define maximization of Sharpe Ratio using principle of duality
#     def f(x, MeanReturns, CovarReturns, RiskFreeRate, PortfolioSize):
#         funcDenomr = np.sqrt(np.matmul(np.matmul(x, CovarReturns), x.T))
#         funcNumer = np.matmul(np.array(MeanReturns), x.T) - RiskFreeRate
#         func = -(funcNumer / funcDenomr)
#         return func
# 
#     #define equality constraint representing fully invested portfolio
#     def constraintEq(x):
#         A = np.ones(x.shape)
#         b = 1
#         constraintVal = np.matmul(A, x.T) - b
#         return constraintVal
# 
#     #define bounds and other parameters
#     xinit = np.repeat(0.33, PortfolioSize)
#     cons = ({'type': 'eq', 'fun': constraintEq})
#     lb = 0
#     ub = 1
#     bnds = tuple([(lb, ub) for x in xinit])
# 
#     #invoke minimize solver
#     opt = minimize(f, x0=xinit, args=(MeanReturns, CovarReturns, RiskFreeRate, PortfolioSize), method='SLSQP',
#                    bounds=bnds, constraints=cons, tol=10 ** -3)
# 
#     return opt


In [2]:
largest_market_cap = pd.read_excel('./Top Companies by Market Cap.xlsx', sheet_name='Sheet1')
largest_market_cap

,Symbol,Company Name,Industry,Market Cap
0,AAPL,Apple Inc.,Consumer Electronics,"2,811.03B"
1,MSFT,Microsoft Corporation,Software - Infrastructure,"2,469.80B"
2,GOOGL,Alphabet Inc.,Internet Content & Information,"1,779.01B"
3,GOOG,Alphabet Inc.,Internet Content & Information,"1,778.79B"
4,AMZN,"Amazon.com, Inc.",Internet Retail,"1,360.20B"
...,...,...,...,...
495,RBLX,Roblox Corporation,Electronic Gaming & Multimedia,19.59B
496,WBA,"Walgreens Boots Alliance, Inc.",Pharmaceutical Retailers,19.51B
497,TW,Tradeweb Markets Inc.,Capital Markets,19.51B
498,ZM,"Zoom Video Communications, Inc.",Software - Application,19.51B


In [3]:
largest_market_cap = largest_market_cap[~largest_market_cap['Symbol'].str.contains('\.')].reset_index(drop=True)
top = largest_market_cap.head(100)
top_symbols = top['Symbol']

In [26]:
# ticker = yf.Ticker('AAPL')
# stock_info = ticker.info
# print(stock_info.get('beta'))

In [4]:
one_year_data = {}
three_year_data = {}
five_year_data = {}
for symbol in top_symbols:
    ticker = yf.Ticker(symbol)
    one_year_data[symbol] = ticker.history(period="1y")
    three_year_data[symbol] = ticker.history(period="3y")
    five_year_data[symbol] = ticker.history(period="5y")

In [7]:
one_year_return_data = {}
three_year_return_data = {}
five_year_return_data = {}
for key, df in one_year_data.items():
    one_year_return_data[key] = calculate_return(one_year_data[key])
    three_year_return_data[key] = calculate_return(three_year_data[key])
    five_year_return_data[key] = calculate_return(five_year_data[key])

one_year_return_data

{'AAPL': 0.2621242345013307,
 'MSFT': 0.5522056896720628,
 'GOOGL': 0.36290150308999786,
 'GOOG': 0.37898479016299175,
 'AMZN': 0.5215656080884936,
 'NVDA': 2.149402126005778,
 'META': 1.9930724137378668,
 'TSLA': 0.28429872505764425,
 'LLY': 0.6638661104507798,
 'V': 0.18992847687950745,
 'UNH': 0.05422366927803753,
 'TSM': 0.23969054976536794,
 'NVO': 0.8442280743276829,
 'WMT': 0.0731406149934084,
 'JPM': 0.1837226054795443,
 'XOM': -0.03907834552893867,
 'JNJ': -0.1214439949544365,
 'MA': 0.17883129491655222,
 'AVGO': 0.9587091973308698,
 'PG': 0.101451548232963,
 'CVX': -0.1820149277903204,
 'ORCL': 0.47914192270564376,
 'HD': 0.010937909043064305,
 'ABBV': -0.05710517527160063,
 'MRK': 0.015101880464667587,
 'COST': 0.11221097468099984,
 'ADBE': 0.7827310591234174,
 'TM': 0.34842234383040144,
 'ASML': 0.17277336561662254,
 'KO': -0.030457821250649393,
 'PEP': -0.048601956062558405,
 'BABA': -0.07138617656243902,
 'SHEL': 0.21276820491490053,
 'CSCO': 0.06184673804357466,
 'BAC': 

In [14]:
def get_deciles(data):
    # Remove None values and sort the data
    filtered_data = {k: v for k, v in data.items() if v is not None}
    sorted_data = sorted(filtered_data.items(), key=lambda x: x[1])

    # Calculate the index for decile
    n = len(sorted_data)
    decile_index = n // 10

    # Get the top and bottom decile
    bottom_decile = dict(sorted_data[:decile_index])
    top_decile = dict(sorted_data[-decile_index:])

    return top_decile, bottom_decile


# Apply the function to each dataset
top_decile_one_year, bottom_decile_one_year = get_deciles(one_year_return_data)
top_decile_three_year, bottom_decile_three_year = get_deciles(three_year_return_data)
top_decile_five_year, bottom_decile_five_year = get_deciles(five_year_return_data)

# Find overlapping keys in the top deciles
overlapping_top_keys = set(top_decile_one_year.keys()) & set(top_decile_three_year.keys()) & set(
    top_decile_five_year.keys())

# Find overlapping keys in the bottom deciles
overlapping_bottom_keys = set(bottom_decile_one_year.keys()) & set(bottom_decile_three_year.keys())

# Print results
print("Top Decile Overlaps:", overlapping_top_keys)
print("Bottom Decile Overlaps:", overlapping_bottom_keys)


Top Decile Overlaps: {'LLY', 'AVGO', 'NVO', 'NVDA'}
Bottom Decile Overlaps: {'HDB', 'BMY', 'NEE'}


# Choosing top-50 and bottom-50 companies

In [30]:
'''''

target_long = pd.DataFrame(['ASML', 'GOOG', 'ACN', 'AAPL', 'SONY',
                            'INTU', 'V', 'MA', 'BX', 'TTE'],
                           columns=['Ticker'])
# TODO
target_short = pd.DataFrame([],
                            columns=['Ticker']
                            )
                            
                            '''''

"''\n\ntarget_long = pd.DataFrame(['ASML', 'GOOG', 'ACN', 'AAPL', 'SONY',\n                            'INTU', 'V', 'MA', 'BX', 'TTE'],\n                           columns=['Ticker'])\n# TODO\ntarget_short = pd.DataFrame([],\n                            columns=['Ticker']\n                            )\n                            \n                            "

In [15]:
# Get all unique keys in the top deciles
all_top_keys = set(top_decile_one_year.keys()) | set(top_decile_three_year.keys()) | set(top_decile_five_year.keys())

# Get all unique keys in the bottom deciles
all_bottom_keys = set(bottom_decile_one_year.keys()) | set(bottom_decile_three_year.keys()) | set(
    bottom_decile_five_year.keys())

all_top_keys

{'AAPL',
 'ADBE',
 'AMAT',
 'AMD',
 'ASML',
 'AVGO',
 'BP',
 'BX',
 'COP',
 'GE',
 'LLY',
 'META',
 'NOW',
 'NVDA',
 'NVO',
 'ORCL',
 'PDD',
 'SHEL',
 'TSLA',
 'XOM'}

In [16]:
all_bottom_keys

{'BA',
 'BABA',
 'BAC',
 'BMY',
 'BUD',
 'CRM',
 'CVX',
 'DIS',
 'HDB',
 'INTC',
 'JNJ',
 'NEE',
 'NKE',
 'PDD',
 'PFE',
 'RTX',
 'T',
 'TMO',
 'UL',
 'UPS',
 'VZ',
 'WFC'}

In [17]:
target_long = pd.DataFrame(list(all_top_keys),
                           columns=['Ticker'])
# TODO
target_short = pd.DataFrame(list(all_bottom_keys),
                            columns=['Ticker'])


# Getting 2-month history

In [18]:
def get_history(target_df: pd.DataFrame, long=True) -> pd.DataFrame:
    history = pd.DataFrame(columns=list(target_long['Ticker']))
    for ticker in target_df['Ticker']:
        # TODO think about the appropriate price column to use
        history[ticker] = yf.Ticker(ticker).history(interval='5m')['High' if long else 'Low']
    return history

In [19]:
history_long = get_history(target_long)
history_long.info()
history_long.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1795 entries, 2023-10-17 14:50:00-04:00 to 2023-11-17 14:50:00-05:00
Data columns (total 20 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ORCL    1795 non-null   float64
 1   NVO     1795 non-null   float64
 2   AMD     1795 non-null   float64
 3   LLY     1795 non-null   float64
 4   BX      1795 non-null   float64
 5   ASML    1794 non-null   float64
 6   GE      1795 non-null   float64
 7   SHEL    1795 non-null   float64
 8   AVGO    1795 non-null   float64
 9   NOW     1795 non-null   float64
 10  TSLA    1795 non-null   float64
 11  AMAT    1795 non-null   float64
 12  META    1795 non-null   float64
 13  ADBE    1795 non-null   float64
 14  PDD     1795 non-null   float64
 15  XOM     1795 non-null   float64
 16  BP      1795 non-null   float64
 17  NVDA    1795 non-null   float64
 18  COP     1795 non-null   float64
 19  AAPL    1795 non-null   float64
dtypes: float64(20)
memory usag

,ORCL,NVO,AMD,LLY,BX,ASML,GE,SHEL,AVGO,NOW,TSLA,AMAT,META,ADBE,PDD,XOM,BP,NVDA,COP,AAPL
Datetime,,,,,,,,,,,,,,,,,,,,
2023-10-17 14:50:00-04:00,109.375000,100.855003,105.199997,604.609985,105.165001,607.690002,110.084999,68.544998,883.099976,568.859985,256.048492,142.107407,322.689789,560.299988,105.040001,111.470001,40.700001,440.734406,125.760002,177.080002
2023-10-17 14:55:00-04:00,109.150002,100.839996,104.800003,604.289978,104.964996,606.525024,109.879997,68.589897,882.059998,568.039917,255.279999,141.919998,322.000000,558.724976,104.879997,111.510002,40.685001,438.600006,125.672302,176.669998
2023-10-17 15:00:00-04:00,109.129997,100.791000,104.680000,604.299988,104.915001,605.974976,109.855003,68.519997,880.940002,568.020020,255.160004,141.839996,322.040009,558.219971,104.929901,111.349998,40.680000,438.179993,125.610001,176.736893
2023-10-17 15:05:00-04:00,109.169296,100.889999,104.723999,604.710022,105.269997,606.501099,110.050003,68.584999,881.659973,567.880005,255.205002,141.910004,322.489990,558.798523,104.949997,111.400002,40.709999,438.229309,125.709999,176.919998
2023-10-17 15:10:00-04:00,109.180000,100.919998,104.779900,605.114990,105.375000,606.560425,110.099998,68.605003,881.691406,567.599976,255.050003,141.820007,322.426910,558.559387,105.010002,111.459999,40.730000,438.519989,125.800003,176.949997


In [40]:
history_short = get_history(target_short)
history_short.dropna(axis=1, inplace=True)
history_short.info()
history_short.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1795 entries, 2023-10-17 15:00:00-04:00 to 2023-11-17 15:00:00-05:00
Data columns (total 22 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   PDD     1795 non-null   float64
 1   BAC     1795 non-null   float64
 2   BMY     1795 non-null   float64
 3   UL      1795 non-null   float64
 4   BA      1795 non-null   float64
 5   NEE     1795 non-null   float64
 6   WFC     1795 non-null   float64
 7   DIS     1795 non-null   float64
 8   BABA    1795 non-null   float64
 9   NKE     1795 non-null   float64
 10  JNJ     1795 non-null   float64
 11  VZ      1795 non-null   float64
 12  BUD     1795 non-null   float64
 13  HDB     1795 non-null   float64
 14  INTC    1795 non-null   float64
 15  TMO     1795 non-null   float64
 16  RTX     1795 non-null   float64
 17  T       1795 non-null   float64
 18  PFE     1795 non-null   float64
 19  CRM     1795 non-null   float64
 20  CVX     1795 non-null   fl

,PDD,BAC,BMY,UL,BA,NEE,WFC,DIS,BABA,NKE,...,BUD,HDB,INTC,TMO,RTX,T,PFE,CRM,CVX,UPS
Datetime,,,,,,,,,,,,,,,,,,,,,
2023-10-17 15:00:00-04:00,104.929901,27.580099,57.189999,48.270000,185.179993,53.950001,41.965000,85.650002,83.900002,102.889999,...,53.950100,58.099602,35.935001,484.600006,73.440002,14.47,32.607498,210.089996,167.690002,156.520004
2023-10-17 15:05:00-04:00,104.949997,27.620001,57.290001,48.325001,185.240005,53.959999,42.050499,85.735001,84.010002,103.019997,...,54.017399,58.119999,35.970001,485.239990,73.519997,14.49,32.695000,210.139999,167.610001,156.850006
2023-10-17 15:10:00-04:00,105.010002,27.620001,57.360001,48.349998,185.289993,54.020000,42.060001,85.769997,84.029999,103.080002,...,54.040001,58.220001,35.990002,485.410004,73.580002,14.51,32.740002,210.130005,167.675003,156.860001
2023-10-17 15:15:00-04:00,105.099998,27.620001,57.369999,48.345001,185.360001,54.009998,42.041302,85.768303,84.040001,103.040001,...,54.005001,58.369999,36.009998,485.619995,73.639999,14.51,32.758999,210.199997,167.610001,156.869995
2023-10-17 15:20:00-04:00,105.150002,27.620001,57.345001,48.334999,185.369995,54.000000,42.040001,85.809998,84.089996,103.050003,...,53.974998,58.369999,36.020000,485.845001,73.669998,14.51,32.719898,210.309998,167.399994,156.970001


# Getting 2-month compound returns

In [41]:
def get_returns(history_df: pd.DataFrame) -> np.ndarray:
    n = len(history_df.columns)
    returns = np.zeros(n, dtype=float)
    for i in range(n):
        # TODO think about the appropriate return calculation
        helper = history_df.iloc[:, i] / history_df.iloc[:, i].shift(1)
        helper = helper.iloc[1:]
        returns[i] = np.prod(helper) - 1
    return returns

In [42]:
returns_long = get_returns(history_long)
print(f'Top companies\' returns: {returns_long}')

Top companies' returns: [ 0.0545362   0.00758514  0.15104563 -0.01773211 -0.00156897  0.12938827
  0.08934005 -0.03457576  0.10963652  0.14775163 -0.08506391  0.04336577
  0.03741146  0.07465647  0.09634428 -0.0560689  -0.12113023  0.12097783
 -0.08396951  0.07183195]


In [43]:
returns_short = get_returns(history_short)
print(f'Bottom companies\' returns: {returns_short}')

Bottom companies' returns: [ 0.0957792   0.08469151 -0.11260881 -0.01947376  0.12177343  0.05838735
  0.02274274  0.09865729 -0.07050061  0.03022646 -0.03548363  0.157694
  0.13614244  0.00215831  0.21093635 -0.03865046  0.08415033  0.09951621
 -0.08425969  0.05578751 -0.13524961 -0.05587152]


# Getting weights

In [48]:
def sharpe_long(weights):
    return -np.dot(returns_long, weights) / np.sqrt(weights @ history_long.cov().to_numpy() @ weights)


def sharpe_short(weights):
    return np.dot(returns_short, weights) / np.sqrt(weights @ history_short.cov().to_numpy() @ weights)

In [49]:
'''''
def get_weights(history_df: pd.DataFrame, long=True) -> np.ndarray:
    n = len(history_df.columns)
    weights = np.ones(n)

    bounds = [(0.0, None)] * n if long else [(None, 0.0)] * n
    constraints = ({'type': 'eq', 'fun': lambda weights: weights.sum() - 1.0})
    optimal_weights = minimize(sharpe_long if long else sharpe_short,
                               weights, 
                               method='SLSQP',
                               constraints=constraints,
                               bounds=bounds
                               ).x

    return optimal_weights
    '''''

"''\ndef get_weights(history_df: pd.DataFrame, long=True) -> np.ndarray:\n    n = len(history_df.columns)\n    weights = np.ones(n)\n\n    bounds = [(0.0, None)] * n if long else [(None, 0.0)] * n\n    constraints = ({'type': 'eq', 'fun': lambda weights: weights.sum() - 1.0})\n    optimal_weights = minimize(sharpe_long if long else sharpe_short,\n                               weights, \n                               method='SLSQP',\n                               constraints=constraints,\n                               bounds=bounds\n                               ).x\n\n    return optimal_weights\n    "

In [50]:
def get_weights(history_df: pd.DataFrame, long=True) -> dict:
    n = len(history_df.columns)
    weights = np.ones(n) if long else np.full(n, -1)

    bounds = [(0.0, 0.10)] * n if long else [(None, 0.0)] * n
    constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1.0})
    optimal_weights = minimize(sharpe_long if long else sharpe_short,
                               weights,
                               method='SLSQP',
                               constraints=constraints,
                               bounds=bounds
                               ).x

    # Create a dictionary with column names as keys and weights as values
    weights_dict = dict(zip(history_df.columns, optimal_weights))
    return weights_dict

In [51]:
weights_long = get_weights(history_long, True)
weights_long_vw = pd.DataFrame(list(weights_long.items()), columns=['Stock', 'Weight'])

weights_long_vw['Weight'] = weights_long_vw['Weight'].apply(lambda x: f"{x * 100:.2f}%")

#print(f'Optimal weights for top companies: {weights_long}')

#print(f'Optimal Sharpe ratio: {-sharpe_long(weights_long)}')

#history_long.columns

print(weights_long_vw)



   Stock  Weight
0   ORCL  10.00%
1    NVO  10.00%
2    AMD  10.00%
3    LLY   0.00%
4     BX  10.00%
5   ASML   0.00%
6     GE  10.00%
7   SHEL  10.00%
8   AVGO   0.00%
9    NOW   0.00%
10  TSLA   0.00%
11  AMAT  10.00%
12  META   0.00%
13  ADBE   0.00%
14   PDD  10.00%
15   XOM  10.00%
16    BP   0.00%
17  NVDA   0.00%
18   COP   0.00%
19  AAPL  10.00%


In [52]:
weights_short = get_weights(history_short, False)
print(f'Optimal weights for bottom companies: {weights_short}')
print(f'Optimal Sharpe ratio: {-sharpe_short(np.array(list(weights_short.values())))}')

Optimal weights for bottom companies: {'PDD': 0.0, 'BAC': -7.771561172376096e-15, 'BMY': -7.549516567451064e-15, 'UL': -3.552713678800501e-15, 'BA': -2.4424906541753444e-15, 'NEE': -9.992007221626409e-15, 'WFC': 0.0, 'DIS': 0.0, 'BABA': -9.325873406851315e-15, 'NKE': -9.992007221626409e-15, 'JNJ': -7.993605777301127e-15, 'VZ': 0.0, 'BUD': 0.0, 'HDB': -6.661338147750939e-16, 'INTC': 0.0, 'TMO': -2.220446049250313e-16, 'RTX': 0.0, 'T': 0.0, 'PFE': -4.440892098500626e-16, 'CRM': -6.217248937900877e-15, 'CVX': 0.0, 'UPS': -1.1102230246251565e-15}
Optimal Sharpe ratio: 0.0019574077106613665
